In [1]:
import pandas as pd
import numpy as np
from gensim.models import Phrases
from gensim.models.word2vec import LineSentence
import spacy
import itertools as it
import os

In [2]:
review = pd.read_csv("/Volumes/Transcend/MDS_Yelp/yelp_review.csv")

In [3]:
business = pd.read_csv("/Volumes/Transcend/MDS_Yelp/yelp_business.csv")

In [4]:
user = pd.read_csv("/Volumes/Transcend/MDS_Yelp/yelp_user.csv")

In [5]:
user = user[user['review_count'] >= 10]

In [6]:
user.head()

,user_id,name,review_count,yelping_since,friends,useful,funny,cool,fans,elite,...,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
0,JJ-aSuM4pCFPdkfoZ34q0Q,Chris,10,2013-09-24,"0njfJmB-7n84DlIgUByCNw, rFn3Xe3RqHxRSxWOU19Gpg...",0,0,0,0,None,...,0,0,0,0,0,0,0,0,0,0
4,4E8--zUZO1Rr1IBK4_83fg,Lisa,11,2012-07-16,None,4,0,0,0,None,...,0,0,0,0,0,0,0,0,1,0
18,jYnkJR3T8yCERXywoVhWYA,Hugo,48,2010-07-06,"hkXekeW_Jj6mIy8r8N7r1Q, dQDpV-VUtwYGqHznuRV-yw...",15,6,2,3,None,...,0,0,0,0,1,1,1,1,2,0
33,tMNryVdaICS7Vpm5ThNKIA,Alex,13,2014-05-27,None,0,0,0,1,None,...,0,0,0,0,0,2,1,1,0,0
44,fV8Yr0c5tFQTQ2SRRJHXHw,Michelle,50,2007-08-22,"HDb4fBWIAQ-foS8qLJty9w, x0hBZsmBTYxhjjx0MShz1A...",43,12,5,1,2009,...,4,0,0,0,9,7,7,7,4,0


In [7]:
review.head()

,review_id,user_id,business_id,stars,date,text,useful,funny,cool
0,vkVSCC7xljjrAI4UGfnKEQ,bv2nCi5Qv5vroFiqKGopiw,AEx2SYEUJmTxVVB18LlCwA,5,2016-05-28,Super simple place but amazing nonetheless. It...,0,0,0
1,n6QzIUObkYshz4dz2QRJTw,bv2nCi5Qv5vroFiqKGopiw,VR6GpWIda3SfvPC-lg9H3w,5,2016-05-28,Small unassuming place that changes their menu...,0,0,0
2,MV3CcKScW05u5LVfF6ok0g,bv2nCi5Qv5vroFiqKGopiw,CKC0-MOWMqoeWf6s-szl8g,5,2016-05-28,Lester's is located in a beautiful neighborhoo...,0,0,0
3,IXvOzsEMYtiJI0CARmj77Q,bv2nCi5Qv5vroFiqKGopiw,ACFtxLv8pGrrxMm6EgjreA,4,2016-05-28,Love coming here. Yes the place always needs t...,0,0,0
4,L_9BTb55X0GDtThi6GlZ6w,bv2nCi5Qv5vroFiqKGopiw,s2I_Ni76bjJNK9yG60iD-Q,4,2016-05-28,Had their chocolate almond croissant and it wa...,0,0,0


In [8]:
review = review[review['stars'] != 3]

In [9]:
business = business[business['city'] == 'Pittsburgh']

In [10]:
business['categories'] = business['categories'].str.lower()
selected_categories = ['restaurant', 'food', 'breakFast', 'pizza', 'burgers', 'diner', 'lunch', 'mexican', 'seafood', 'italian']
selected_categories = selected_categories + ['chinese', 'indian', 'korean', 'asian', 'sushi', 'brunch', 'cafe', 'thai']
selected_categories = selected_categories + ['sandwich', 'bake', 'bagels']

In [54]:
business_restaurant = business[business['categories'].str.contains('restaurant')]
business_selected = pd.DataFrame(columns = list(business.columns.values))
for i in range(0,len(selected_categories)):
    df = business[business['categories'].str.contains(selected_categories[i])]
    business_selected = business_selected.append(df)
business_selected = business_selected.drop_duplicates()
business_selected.shape

(2879, 13)

In [55]:
business_selected = business_selected[business_selected['review_count'] > 100]
business_selected.shape

(374, 13)

In [13]:
user_business = review[['user_id', 'business_id']]
user_business.head()

,user_id,business_id
0,bv2nCi5Qv5vroFiqKGopiw,AEx2SYEUJmTxVVB18LlCwA
1,bv2nCi5Qv5vroFiqKGopiw,VR6GpWIda3SfvPC-lg9H3w
2,bv2nCi5Qv5vroFiqKGopiw,CKC0-MOWMqoeWf6s-szl8g
3,bv2nCi5Qv5vroFiqKGopiw,ACFtxLv8pGrrxMm6EgjreA
4,bv2nCi5Qv5vroFiqKGopiw,s2I_Ni76bjJNK9yG60iD-Q


##### filtered Users to selected_categories and Pittsburgh

In [56]:
x = list(user_business['user_id'])

In [57]:
len(x)

107463

In [65]:
review_business = pd.merge(review[['user_id', 'business_id']], business_selected[['business_id', 'stars']], on = 'business_id')

In [66]:
review_business.shape

(68391, 3)

In [73]:
user_business = pd.merge(user[['user_id', 'review_count']], review_business[['user_id', 'business_id']], on = 'user_id')

In [74]:
user_business.shape

(50337, 3)

In [75]:
user_business = user_business[['user_id', 'business_id']] #filtered to selected_categories and Pittsburgh
user_business.head()

,user_id,business_id
0,86M8RPM6g0QFCOLNNfMMjg,vNQkym5qZ-E-4AJ4IQ5JEg
1,86M8RPM6g0QFCOLNNfMMjg,uEzodpjckhsVQFfsXpFR0w
2,4H_sZ7c_zYuhBMJPDQZIyg,TFdLM8R_7Pf1xVeBNKl0hg
3,BH_8hwrP1wwRHVTE8O3LEg,hsDkxhc-_YYsVqAeQBJJjg
4,vMrgHrPV6sidvTMQLyJtXQ,Hdnx6cZBo0JfZopnQDWVYg


In [82]:
user_business_matrix = user_business.groupby(['user_id','business_id'], as_index = False).size().unstack(fill_value=0)

In [85]:
user_business_matrix.head()

business_id,-7H-oXvCxJzuT42ky6Db0g,-7PX_FOoCwktlunImRyZdg,-WVF5DKZeEWlMK8Cucto-A,-d8gt_XhhUQh2CxlWJe-xw,-httBl6DSZiUtDX4SQWtag,-i3pCgQi_Y9NiSSWs6G7bw,-j0CbYvlwuJ3Tfx__X38gA,04kZ5CSh6oKhI5huU5bLdg,08t3_HjbpLyPeuft6eoa5A,0PCBt3JKD6IooicImKNBzA,...,yAWSOUNFcfobOT_FRg_7Gg,yPcAdikNrXEsfbHNBQUjkQ,yZ_wpRr4Svw8YRZ9voCgIw,ysJo5Jdo29XOBCnKrbUKWg,z15C-dJodOlPO8bMBl6F8g,z2RiCl_iU-8bM4Lj1zNL8g,zMfZLd5_h5W8ZVxTNoeJyQ,ziJsGjXvidzZWC1I0-SOSg,zuUWG7OdMv6awFRYhEYT5Q,zxSfGIhK3hH3vVz_pS5eaA
user_id,,,,,,,,,,,,,,,,,,,,,
--CH8yRGXhO2MmbF-4BWXg,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
--EMqnd727rtC0G5Oc-Mrg,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
--RBfYfIpx44V5Kux2fPFA,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
--cO5bedOWodnt8UgamQNA,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
--mQ4S5h1tXzvE9VDYVwdQ,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [78]:
x = user_business_matrix.sort_values(by = '-7H-oXvCxJzuT42ky6Db0g', ascending = False)
x.head()

business_id,-7H-oXvCxJzuT42ky6Db0g,-7PX_FOoCwktlunImRyZdg,-WVF5DKZeEWlMK8Cucto-A,-d8gt_XhhUQh2CxlWJe-xw,-httBl6DSZiUtDX4SQWtag,-i3pCgQi_Y9NiSSWs6G7bw,-j0CbYvlwuJ3Tfx__X38gA,04kZ5CSh6oKhI5huU5bLdg,08t3_HjbpLyPeuft6eoa5A,0PCBt3JKD6IooicImKNBzA,...,yAWSOUNFcfobOT_FRg_7Gg,yPcAdikNrXEsfbHNBQUjkQ,yZ_wpRr4Svw8YRZ9voCgIw,ysJo5Jdo29XOBCnKrbUKWg,z15C-dJodOlPO8bMBl6F8g,z2RiCl_iU-8bM4Lj1zNL8g,zMfZLd5_h5W8ZVxTNoeJyQ,ziJsGjXvidzZWC1I0-SOSg,zuUWG7OdMv6awFRYhEYT5Q,zxSfGIhK3hH3vVz_pS5eaA
user_id,,,,,,,,,,,,,,,,,,,,,
6z9ERfBvRF3HOXLNTNkdpg,1,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
gWLeQBH2tgHk6OsBAUzNxA,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Z_C2bWE-48ivuTEoA635dA,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
BH4SB-msnsZZtkuL2BpZWw,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
J9An8BT-KlKSZdiyKoRWUA,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [46]:
y = user_business.groupby(['business_id']).agg(['count'])

In [47]:
y.head()

,user_id
,count
business_id,
--cgVkbWTiga3OYTkymKqA,2
--cjBEbXMI2obtaRHNSFrA,28
--ujyvoQlwVoBgMYtADiLA,39
--wIGbLEhlpl_UeAIyDmZQ,9
-0nEqS3_A_nnUut4DrEPEg,1


In [48]:
y = y.reset_index()
y.columns = y.columns.droplevel(0)
y.head()

,,count
0,--cgVkbWTiga3OYTkymKqA,2
1,--cjBEbXMI2obtaRHNSFrA,28
2,--ujyvoQlwVoBgMYtADiLA,39
3,--wIGbLEhlpl_UeAIyDmZQ,9
4,-0nEqS3_A_nnUut4DrEPEg,1


In [49]:
y.columns.values = ['business_id', 'business_rev_count']

Index(['', 'count'], dtype='object')

In [87]:
#user_business_matrix['user_id'] = user_business_matrix.index
#user_business_matrix.reset_index(level=0, inplace= True)
user_business_matrix.head()

business_id,-7H-oXvCxJzuT42ky6Db0g,-7PX_FOoCwktlunImRyZdg,-WVF5DKZeEWlMK8Cucto-A,-d8gt_XhhUQh2CxlWJe-xw,-httBl6DSZiUtDX4SQWtag,-i3pCgQi_Y9NiSSWs6G7bw,-j0CbYvlwuJ3Tfx__X38gA,04kZ5CSh6oKhI5huU5bLdg,08t3_HjbpLyPeuft6eoa5A,0PCBt3JKD6IooicImKNBzA,...,yPcAdikNrXEsfbHNBQUjkQ,yZ_wpRr4Svw8YRZ9voCgIw,ysJo5Jdo29XOBCnKrbUKWg,z15C-dJodOlPO8bMBl6F8g,z2RiCl_iU-8bM4Lj1zNL8g,zMfZLd5_h5W8ZVxTNoeJyQ,ziJsGjXvidzZWC1I0-SOSg,zuUWG7OdMv6awFRYhEYT5Q,zxSfGIhK3hH3vVz_pS5eaA,user_id
user_id,,,,,,,,,,,,,,,,,,,,,
--CH8yRGXhO2MmbF-4BWXg,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,--CH8yRGXhO2MmbF-4BWXg
--EMqnd727rtC0G5Oc-Mrg,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,--EMqnd727rtC0G5Oc-Mrg
--RBfYfIpx44V5Kux2fPFA,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,--RBfYfIpx44V5Kux2fPFA
--cO5bedOWodnt8UgamQNA,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,--cO5bedOWodnt8UgamQNA
--mQ4S5h1tXzvE9VDYVwdQ,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,--mQ4S5h1tXzvE9VDYVwdQ


In [92]:
user_business_matrix.to_csv('/Volumes/Transcend/MDS_Yelp/user_business_matrix.csv')

In [93]:
temp = user_business_matrix.sum(axis = 1)

In [97]:
temp = temp.sort_values(ascending = False)
temp.head(100)

user_id
4wp4XI9AxKNqJima-xahlg    185
rCWrxuRC8_pfagpchtHp6A    120
135DbbQnr3BEkQbBzZ9T1A    118
4m9NXICYBC5i9t4aTt-I6w    109
2jKzO_01d12oiu-2bOYcYg    104
6Ki3bAL0wx9ymbdJqbSWMA    101
8AwcaBJjiMpQ__FPxktwwQ    101
hLVq7VSJBHZwqurwWoCmpg     94
cCke3VtFLtqgzkgYSrSE2g     91
DeXKbQYNx52OlOizobOLJw     88
Lfv4hefW1VbvaC2gatTFWA     87
H7pj7sbXY3N-WSEwa-JfpA     86
NnrCwpS1i5mZtkGGzmIk0g     81
2ZZNOoMQ7WFvFoxDUFw1wg     71
Ic6-gs1_FjrWGx6JIr95Mw     70
M9TjYmTgHayJ22cNkyvW_g     70
PD3X7S70zXHeOx1NjeicjQ     67
BkMqpJikNc3r5itc-ui6ww     67
BTKtAXjPHjH5B3xuT4uiIw     67
SqVUNVeYJblyoUz4e-Fiqw     67
57tf_RmT5o3BiFj2gpV-Jg     67
bu1mtWwxuayzV6b3VScWLw     67
d0D7L-vfQDIADolnPAcb9A     66
3DltS5Wr9MOntnc8StBFBA     65
kXt6tPSGLWhWTiXO7XPjuA     63
3bo1FXvQnxw2hmO0CHH4Nw     62
-Vu7L3U7-kxDyY1VHxw3zw     59
EWkQxKg-KOzujC38Z_40wg     57
JiPMk9WmbJu-VfTRAKpZpw     56
iJp3Wu-pYl7osiYqz1Gv7A     56
                         ... 
H_09-Wr5JcN5hpp5RnSLTA     41
Mrudkq5duCOoiuhL61ZBcQ     40
fh